### 0. Import Libs

In [1]:
# kernel must be msd2
import sys;sys.path.append('../')
import constant as c
import os
import pandas as pd
import warnings; warnings.filterwarnings("ignore")
from util.epsg_transform import *
from shapely.geometry import *

### 1. Get Metro Data

In [6]:
# get metro origin/destination df
metro_folder_path = os.path.join(c.SSD_PATH,'metro')
metro_fname = '서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv'
metro_path = os.path.join(metro_folder_path, metro_fname)
metro = pd.read_csv(metro_path, encoding = 'euc-kr')
yyyymms = ['202001', '202101', '202201']
metro = metro[metro['사용월'].isin([202001, 202101, 202201])]
metro = metro.drop(columns='작업일자')
metro['지하철역'] = metro['지하철역'].str.replace(r'\s*\([^)]*\)', '')

# get metro location df
metroloc_fname = 'metroloc.csv'
metroloc_path = os.path.join(metro_folder_path, metroloc_fname)
metroloc = pd.read_csv(metroloc_path, encoding = 'utf-8')
metroloc['name'] = metroloc['name'].apply(lambda x:x.strip('"'))
metroloc['line'] = metroloc['line'].apply(lambda x:x.strip('"'))
metroloc['line'] = metroloc['line'].str.lstrip('0')
metroloc = metroloc.rename(columns = {'lat': 'y', 'lng': 'x', 'line': '호선명', 'name': '지하철역'})
metroloc['x'] = pd.to_numeric(metroloc['x'], errors='coerce')
metroloc['y'] = pd.to_numeric(metroloc['y'], errors='coerce')

# merge two dfs
df_metro = pd.merge(metro, metroloc, on = ['호선명', '지하철역'])
df_metro['사용월'] = df_metro['사용월'].astype(str)
df_metro['year'] = df_metro['사용월'].str[:4]


x_5179, y_5179 = coord_tranformation(list(df_metro.x), list(df_metro.y), "epsg:4326", "epsg:5179")
df_metro['x_5179'] = x_5179
df_metro['y_5179'] = y_5179

df_metro = df_metro.rename(columns=c.DF_METRO_COL_AMEND_DICT)
df_metro.to_csv('../asset/preprocess/features/df_metro.csv')


In [3]:
# Read df_metro
df_metro = pd.read_csv('../asset/preprocess/features/df_metro.csv', index_col = 0)